# The quintic Ornstein-Uhlenbeck volatility model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import BlackScholes as bs
import variance_curve as vc
import risk_free_rates as rf
import implied_q as iq
import time
import scipy

from scipy.optimize import least_squares as ls

In [2]:
S0 = 4017.8
t0 = "23 Jan 2023"

IV_df = pd.read_csv("hist_spx.csv")
moneyness = np.array([80.0,90.0,95.0,97.5,100.0,102.5,105.0,110.0,120.0])
maturities = np.array(IV_df['Exp Date']).flatten()
IV = np.array(IV_df.drop(columns = 'Exp Date'))/100.

K = moneyness*S0/100

## SPX options by Monte Carlo with antithetic and control variates

In [3]:
def horner_vector(poly, n, x):
    #Initialize result
    result = poly[0].reshape(-1,1)
    for i in range(1,n):
        result = result*x + poly[i].reshape(-1,1)
    return result

In [4]:
def dW(n_steps,N_sims):
    w = np.random.normal(0, 1, (n_steps, N_sims))
    #Antithetic variates
    w = np.concatenate((w, -w), axis = 1)
    return w

In [5]:
def price(S, K, r, q, T, opt):
    # opt stands for the option type: True = call, False = put
    N = len(K)
    P = np.zeros(N)
    for i in range(N):
        if opt[i]:
            P[i] = np.mean(np.maximum(S-K[i],0)*np.exp(-(r-q)*T))
        else:
            P[i] = np.mean(np.maximum(K[i]-S,0)*np.exp(-(r-q)*T))
    return P

In [6]:
def local_reduction(rho,H,eps,T,a_k_part,S0,strike_array,n_steps,N_sims,w1,r,q):
    
    spine_k_order = 3
    
    eta_tild = eps**(H-0.5)
    kappa_tild = (0.5-H)/eps
    
    a_0,a_1,a_3,a_5 = a_k_part
    a_k = np.array([a_0,a_1,0,a_3,0,a_5])

    dt = T/n_steps
    tt = np.linspace(0., T, n_steps + 1)

    exp1 = np.exp(kappa_tild*tt)
    exp2 = np.exp(2*kappa_tild*tt)

    diff_exp2 = np.concatenate((np.array([0.]),np.diff(exp2)))
    std_vec = np.sqrt(diff_exp2/(2*kappa_tild))[:,np.newaxis] #to be broadcasted columnwise 
    exp1 = exp1[:,np.newaxis] 
    X = (1/exp1)*(eta_tild*np.cumsum(std_vec*w1, axis = 0)) 
    Xt = np.array(X[:-1])
    del X
    
    tt = tt[:-1]
    std_X_t = np.sqrt(eta_tild**2/(2*kappa_tild)*(1-np.exp(-2*kappa_tild*tt)))
    n = len(a_k)
    
    cauchy_product = np.convolve(a_k,a_k)
    normal_var = np.sum(cauchy_product[np.arange(0,2*n,2)].reshape(-1,1)*std_X_t**(np.arange(0,2*n,2).reshape(-1,1))*\
        scipy.special.factorial2(np.arange(0,2*n,2).reshape(-1,1)-1),axis=0)
    
    f_func = horner_vector(a_k[::-1], len(a_k), Xt)
        
    del Xt
    
    fv_curve = vc.variance_curve(tt).reshape(-1,1)

    volatility = f_func/np.sqrt(normal_var.reshape(-1,1))
    del f_func
    volatility = np.sqrt(fv_curve)*volatility
    
    logS1 = np.log(S0)
    for i in range(w1.shape[0]-1):
        logS1 = logS1 - 0.5*dt*(volatility[i]*rho)**2 + np.sqrt(dt)*rho*volatility[i]*w1[i+1] + rho**2*(r-q)*dt
    del w1
    ST1 = np.exp(logS1)
    del logS1 

    int_var = np.sum(volatility[:-1,]**2*dt,axis=0)
    Q = np.max(int_var)+1e-9
    del volatility
    X = (bs.BSCall(ST1, strike_array.reshape(-1,1), T, r, q, np.sqrt((1-rho**2)*int_var/T))).T
    Y = (bs.BSCall(ST1, strike_array.reshape(-1,1), T, r, q, np.sqrt(rho**2*(Q-int_var)/T))).T
    del int_var
    eY = (bs.BSCall(S0, strike_array.reshape(-1,1), T, r, q, np.sqrt(rho**2*Q/T))).T
    
    c = []
    for i in range(strike_array.shape[0]):
        cova = np.cov(X[:,i]+10,Y[:,i]+10)[0,1]
        varg = np.cov(X[:,i]+10,Y[:,i]+10)[1,1]
        if (cova or varg)<1e-8:
            temp = 1e-40
        else:
            temp = np.nan_to_num(cova/varg,1e-40)
        temp = np.minimum(temp,2)
        c.append(temp)
    c = np.array(c)
    
    call_mc_cv1 = X-c*(Y-eY)
    del X
    del Y
    del eY
    
    return np.average(call_mc_cv1,axis=0)

In [10]:
start_time_all = time.time()
nr = len(maturities); nc = len(K);
model_vol = np.zeros([nr,nc]); model_param = np.zeros([nr,7])
inp = np.array([-0.65, 0.2, 0.02, 1, 0.01, 0.02, 0.05]) # Parameter array [rho, H, eps, a0, a1, a3, a5]
bnds = ([-0.999, -5, 1e-10, 0, 0, 0, 0],[-1e-9, 0.5, 0.5, np.inf, np.inf, np.inf, np.inf])
N = 12500; n = 1000;
t = np.zeros(nr)

w = np.concatenate((np.zeros([1,N*2]), dW(n, N)))

for i in range(nr):
    
    start_time = time.time()
    
    vol = IV[i]; T = maturities[i];    
    r = rf.r(T); q = iq.q(T);
    
    if i > 20:
        bnds = ([-0.999, -0.75, 1e-10, 0, 0, 0, 0],[-1e-9, 0.5, 0.5, np.inf, np.inf, np.inf, np.inf])
    if i > 26:
        bnds = ([-0.999, -0.25, 1e-10, 0, 0, 0, 0],[-1e-9, 0.5, 0.5, np.inf, np.inf, np.inf, np.inf])
    if i == 31:
        bnds = ([-0.999, -0.1, 1e-10, 0, 0, 0, 0],[-1e-9, 0.5, 0.5, np.inf, np.inf, np.inf, np.inf])
    
    def h(x):
    
        np.random.seed(0)
        
        rho, H, eps, a0, a1, a3, a5 = x
        
        a_k = np.array([a0,a1,a3,a5])
        P = local_reduction(rho, H, eps, T, a_k, S0, K, n, N, w, r, q)

        return bs.BSImpliedVol(S0, K, T, r, q, P, Option_type = 1, toll = 1e-5)

    def f(x):
        return h(x) - vol

    result = ls(f, inp, bounds = bnds, max_nfev = 10, ftol = 1e-10, gtol = 1e-10, xtol = 1e-10)
    model_param[i,:] = result.x
    
    t[i] = time.time() - start_time

    print(f'Iteration: {i}\t Elapsed time: {t[i]: .0f} s')
    
    model_vol[i,:] = h(result.x)

total_time = (time.time() - start_time_all)/60

print(f'Total execution time: {total_time: .0f} minutes')

print(f'\nMean relative error: {np.mean(np.abs((model_vol-IV)/IV))*100: .4f}%')

Iteration: 21	 Elapsed time:  122 s
Iteration: 22	 Elapsed time:  157 s
Iteration: 23	 Elapsed time:  155 s
Iteration: 24	 Elapsed time:  138 s
Iteration: 25	 Elapsed time:  138 s
Iteration: 26	 Elapsed time:  170 s
Iteration: 27	 Elapsed time:  138 s
Iteration: 28	 Elapsed time:  139 s
Iteration: 29	 Elapsed time:  106 s
Iteration: 30	 Elapsed time:  138 s
Iteration: 31	 Elapsed time:  172 s
Total execution time:  27 minutes

Mean relative error:  2.2314%


In [ ]:
print('3.8522%')

In [11]:
df = pd.DataFrame(model_param, columns = ["rho", "H", "eps", "a0", "a1", "a3", "a5"])
df.to_csv("quintic_local_parameters.csv", index = False)

In [12]:
db = pd.DataFrame(model_vol)
db.to_csv("quintic_local_iv.csv", index = False)